In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import imageio
import os
import math

In [ ]:
##### user input for grid size and number of generations
rows = int(input("How many rows of cells? "))
cols = int(input("How many columns of cells? "))
itns = int(input("How many iterations? "))

filenames = []

# set grid size
q = (rows, cols)

# function to get neighbours of a cell
neighbors = lambda x, y : [(x2, y2) for x2 in range(x-1, x+2)
                                   for y2 in range(y-1, y+2)
                                   if (-1 < x <= rows and
                                       -1 < y <= cols and
                                       (x != x2 or y != y2) and
                                       (0 <= x2 <= rows-1) and
                                       (0 <= y2 <= cols-1))]

# ODE representing u and v expression
def exp(x, t):
    u, v = x
    dudt = (p0 * ((p1+(R1 * (S**2 / (K1**2 + S**2))))/(1+p1+(R1 * (S**2 / (K1**2 + S**2))) + k*v**2))) - du*u
    dvdt = (p2 * ((p3+(R2 * (S**2 / (K2**2 + S**2))))/(1+p3+(R2 * (S**2 / (K2**2 + S**2))) + k*u**2 + k*v**2))) - dv*v
    dxdt = [dudt, dvdt]
    return dxdt

# optimal parameters:
p0 = 288.9950 * 100
p1 = 50.9608 
p2 = 167.1198 * 100
p3 = p1
k = 0.06
R1 = 295.5976
R2 = 295.2462
K1 = 10.0259
K2 = 143.5996
du = 1 * 100
dv = 0.12 * 100
# time scale for integration of function exp
t = np.linspace(0,10,301)

# function to calculate u experienced by cell, based on diffusion
def udiffuse(t2):
    u_int = u_ext - ((u_ext - u_int0) * (1 - k_u * t2))
    return u_int
k_u = 0.05 # diffusion constant

# function to calculate signal based on total u and internal v
def get_sig_ON(l):
    if l >= 310:
        S = (18.628 * l) - 5784.8
    else:
        S = (0.9659 * l) - 297.21
    if S >= 100:
        S = 100
    elif S <= 0:
        S = 0
    return S

def get_sig_OFF(l):
    if l >= 310:
        S = (20.444 * l) - 6354.2
    else:
        S = (0.4829 * l) - 148.61
    if S >= 100:
        S = 100
    elif S <= 0:
        S = 0
    return S

# calculation of initial expression values for ON and OFF cells
S = 0 # OFF cells
exps_off = odeint(exp, [200,0], t)
initial_u_OFF = (exps_off[:, 0][100])
initial_v_OFF = (exps_off[:, 1][100])

S = 20 # ON cells
exps_on = odeint(exp, [200,0], t)
initial_u_ON = (exps_on[:, 0][100])
initial_v_ON = (exps_on[:, 1][100])


# choose initial grid configuration:

grid = np.zeros(q) # initialise grid of zeros for adding custom configs

on_cells = [(4,3),(4,4),(4,5)] # set indices of blinker
# # on_cells = [(3,1),(3,2),(3,3),(2,3),(1,2)] # or set indices of glider

for c in on_cells: 
    grid[c] = 1 # set these chosen cells to ON in initial grid

# grid = np.random.randint(2, size=q) # or switch off all the code above and just use this to generate random config


# create internal expression grid for u (first) and v (second)
exp_internal = (np.zeros(q), np.zeros(q))
# exp_internal[0] = internal u exp
# exp_internal[1] = internal v exp
for i in range(rows):
    for j in range(cols):
        if grid[i,j] == 1:
            exp_internal[0][i,j] = initial_u_ON
            exp_internal[1][i,j] = initial_v_ON
        elif grid[i,j] == 0:
            exp_internal[0][i,j] = initial_u_OFF
            exp_internal[1][i,j] = initial_v_OFF

# print("u on", initial_u_ON)
# print("u off", initial_u_OFF)
# print("v on", initial_v_ON)
# print("v off", initial_v_OFF)
            
# save the grid with INITIAL u exp values at t=0
# exp_initial = exp_internal[0] 

# outer loop for time (i.e. iterates through each generation)
for g in range(itns):
    exp_internal_temp = (np.zeros(q), np.zeros(q))

    filename = f'{g}.png'
    filenames.append(filename)
    
    plt.figure()
    plt.imshow(grid)
    plt.savefig(filename)
    plt.show()
    plt.close()
#     print(exp_internal[0], "\n")

# inner loop that iterates through each cell in grid
    for i in range(rows):
        for j in range(cols):
            external_u = 0
            ON_neigh_count = 0
            neigh_count = 0
            
            for n in neighbors(i,j): # function to calculate neighbours of each cell
                neigh_count += 1 # count total number of neighbours
                if grid[n] == 1:  # count number of alive neighbours
                    ON_neigh_count += 1
                u_val = exp_internal[0][n] # retrieve u expression of each neighbour
                external_u += u_val # u_val = external u expression from each neighbour
#                 print("For cell", (i,j), "Neighbour indexes:", n)
            
#             print("ON neighbours:", ON_neigh_count)
            
            u_ext = external_u / neigh_count # calculate average u_ext
            
            # setting initial u_exp to u_exp in the previous iteration
            u_int0 = (exp_internal[0][i,j]) # this causes expression levels to increase with each iteration, and as a result the calculated parameter values for calculating S become wacky after first itn 
        
#             print("initial:", u_int0)
            
            v_int = (exp_internal[1][i,j])
            
            # calculate total u in cell based on diffusion between surrounding cells and store in variable
            u_tot = udiffuse(1)
#             print("after diffusion:", u_tot)
            
            # calculate signal that cell recieves due to expression of u and v
            l = u_tot + (2.815 * v_int)
#             print("u + av:", l)
            
            if grid[i,j] == 1:
                S = (get_sig_ON(l))
            elif grid[i,j] == 0:
                S = (get_sig_OFF(l))
                
#             print("calculated signal:", S)
            
            # set initial u and v values based on which cell it is
            x0 = [u_tot, v_int] # initial u_exp is now the calculated u_tot after allowing diffusion to take place
            # x0_reshaped = x0.reshape(2,)
            
            # integrate ODEs with this S value to obtain new expression values
            new_exps = odeint(exp, x0, t)
            new_u = (new_exps[:, 0][300])
#             print("new exp:", new_u, "\n")
            new_v = (new_exps[:, 1][300])
            
            # store new expression values in a temporary grid
            exp_internal_temp[0][i,j] = new_u
            exp_internal_temp[1][i,j] = new_v
            
            # check new expresssion values against threshold to calculate new state
            if new_u > 162.5:
                grid_new[i,j] = 1
            else: grid_new[i,j] = 0
            
    exp_internal = exp_internal_temp # update the internal expression grid
    grid = grid_new 
#     print(exp_external)
            
with imageio.get_writer('gif_stage4.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
for filename in set(filenames):
    os.remove(filename)  